In [4]:
import requests
import json
from collections import defaultdict

In [15]:
f = open('propublica-api-key.txt', 'r')
text = f.read()
api_key = text.strip()
headers = {'X-API-Key': api_key}

In [16]:
def get_senators():
    """ get senator information """
    url = 'https://api.propublica.org/congress/v1/115/senate/members.json'
    response = requests.get(url, headers=headers).text  # returns str
    senators = json.loads(response)['results'][0].get('members')
    members = []
    for senator in senators:
        d = {'id': senator['id'], 'first_name': senator['first_name'], 'last_name': senator['last_name'],
             'gender': senator['gender'], 'party': senator['party'], 'twitter_account': senator['twitter_account'],
             'state': senator['state']}
        members.append(d)
    return members

In [17]:
def get_recent_bills():
    all_bills = []
    url = 'https://api.propublica.org/congress/v1/both/votes/recent.json'
    response = requests.get(url, headers=headers).text  # returns str
    bills = json.loads(response)['results']['votes']  # create json object from str
    for vote in bills:
        if 'bill_id' in vote['bill']:
            d = {'bill_id': vote['bill']['bill_id'], 'chamber': vote['chamber'], 'roll_call': vote['roll_call'],
                        'title': vote['bill']['title'], 'date': vote['date'], 'result': vote['result'],
                        'd_yes': vote['democratic']['yes'], 'd_no': vote['democratic']['no'],
                        'r_yes': vote['republican']['yes'], 'r_no': vote['republican']['no'],
                        'i_yes': vote['independent']['yes'], 'i_no': vote['independent']['no']}
            all_bills.append(d)
        else:
            continue
    return all_bills

In [18]:
def get_roll_call():
    vote_tally = []
    for vote in all_bills:
        roll_call_results = defaultdict(list)
        roll_call_id = vote['roll_call']
        url = f'https://api.propublica.org/congress/v1/115/senate/sessions/1/votes/{roll_call_id}.json'
        response = requests.get(url, headers=headers).text
        representatives = json.loads(response)['results']['votes']['vote']['positions']
        for rep in representatives:
            d = {'member_id': rep['member_id'], 'name': rep['name'], 'party': rep['party'],
                     'state': rep['state'], 'vote': rep['vote_position']}
            roll_call_results[roll_call_id].append(d)
        vote_tally.append(roll_call_results)
    return vote_tally

In [19]:
all_members = get_senators()
all_bills = get_recent_bills()
votes = get_roll_call()

In [21]:
with open('senators.json', 'w') as outfile:  
    json.dump(all_members, outfile)

In [22]:
with open('bills.json', 'w') as outfile:  
    json.dump(all_bills, outfile)

In [23]:
with open('votes.json', 'w') as outfile:  
    json.dump(votes, outfile)